In [ ]:
# main.py

from agent_system.workflows.industry_research import run_industry_research

In [ ]:
if __name__ == "__main__":
    result = run_industry_research(
        {
            "industry": "人工智能",
            "target_year": "2025",
            "focus":"全产业链分析",
            "province":"上海市"
        }
    )
    print(result)

In [ ]:
切换到目录 cd /d D:\python\AI agent\investment_agent_crewai
在anaconda prompt中相应路径执行命令 streamlit run app.py

### 改进方案

In [ ]:
这是一个非常典型且令人抓狂的错误。根据您提供的信息和报错内容 `'\n "need_revision"'`，以及您反复修改 `reviewer_parser.py` 却无效的现象，问题主要锁定在以下 **3 个关键点**：

1. **代码未生效（缓存问题）**：Python 的 `__pycache__` 缓存了旧版解析器，导致您新写的“保底逻辑”根本没执行。
2. **代码逻辑缺失（变量未定义）**：在您上传的 `industry_research.py` 中，**Phase 5 部分缺少了运行 Crew 的关键代码**，导致 `review_result` 变量不存在或内容错误。
3. **架构缺陷**：依靠“大模型输出 JSON -> 正则解析”本身就不稳定。**最好的解决方式是使用 CrewAI 的结构化输出功能 (`output_pydantic`)**，彻底抛弃手动解析。

以下是分步解决方案：

---

### 第一步：修复 `industry_research.py` 中的致命遗漏

在您上传的 `industry_research.py` 文件中，**Phase 5** 部分定义了 `review_crew` 后，直接就调用了解析函数，**漏掉了 `.kickoff()` 这一行**。这会导致变量 `review_result` 未定义（NameError）或者沿用了前面步骤的旧变量。

请检查并修改 `agent_system/workflows/industry_research.py` 约 370 行附近：

```python
    review_crew = Crew(
        agents=[reviewer],
        tasks=[review_task],
        process=Process.sequential,
        verbose=True
    )

    # ❌ 原代码可能漏了这一行，或者变量名写错
    # review_struct = parse_reviewer_output(review_result) 

    # ✅ 【必需补全】运行 Crew 并获取结果字符串
    review_result_obj = review_crew.kickoff()
    review_result = str(review_result_obj)  # 强制转为字符串

    # 然后再解析
    review_struct = parse_reviewer_output(review_result)

```

---

### 第二步：强制清理缓存（解决“修改无效”的根源）

您修改了 `reviewer_parser.py` 但报错依旧，这通常是因为 Python 仍在运行旧的字节码文件 (`.pyc`)。

**请务必执行以下操作：**

1. **停止程序**：在终端按 `Ctrl + C`。
2. **删除缓存**：
* 删除 `agent_system/postprocess/__pycache__` 文件夹。
* 或者直接删除项目根目录下所有的 `__pycache__` 文件夹。


3. **重启**：重新运行 `streamlit run app.py`。

---

### 第三步：【进阶方案】抛弃手动解析，使用 Pydantic 结构化输出（最佳实践）

手动写正则去匹配 JSON 是最痛苦且不稳定的。**更好的解决方式**是利用 CrewAI 对 Pydantic 的原生支持，让大模型强制输出符合 Python 类定义的结构。这样您就不需要写 parser 了。

#### 1. 新建结构定义文件

在 `agent_system/schemas/reviewer_output.py`（需新建）中定义输出格式：

```python
from typing import List, Optional
from pydantic import BaseModel, Field

class RevisionTask(BaseModel):
    chapter: str = Field(..., description="需要修改的章节标题")
    section: Optional[str] = Field(None, description="需要修改的小节")
    issue: str = Field(..., description="存在的问题说明")
    rewrite_requirement: str = Field(..., description="具体的补写指令")

class ReviewerOutput(BaseModel):
    need_revision: bool = Field(False, description="是否需要触发局部补写机制")
    revision_tasks: List[RevisionTask] = Field(default_factory=list, description="具体的修改任务列表")

```

#### 2. 修改 `industry_research.py` 的 Task 定义

修改 **Phase 5** 的 `review_task`，移除 `parse_reviewer_output`，直接让 Task 输出对象：

```python
# 引入定义
from agent_system.schemas.reviewer_output import ReviewerOutput

# ...

    # 修改 Review Task
    review_task = Task(
        description=REVIEWER_PROMPT.format(report=draft_report),
        # 🔥【关键修改】直接指定 output_pydantic，CrewAI 会自动处理 JSON 解析和重试
        output_pydantic=ReviewerOutput, 
        agent=reviewer
    )

    review_crew = Crew(
        agents=[reviewer],
        tasks=[review_task],
        verbose=True
    )

    # 运行
    result_obj = review_crew.kickoff()
    
    # 🔥【关键修改】结果直接就是 Pydantic 对象，不仅有属性，还能转 dict
    # result_obj.pydantic 包含了我们要的数据
    if result_obj.pydantic:
        review_data = result_obj.pydantic
        # 直接使用对象属性，再也不用担心 Key Error
        if review_data.need_revision:
            print("🔁 Reviewer 触发局部补写机制")
            for task in review_data.revision_tasks:
                # task.chapter, task.issue ...
                pass
    else:
        # 降级处理：如果 Pydantic 解析失败（极少情况），result_obj.raw 包含原始文本
        print("⚠️ 结构化输出失败，跳过审核")
        review_data = ReviewerOutput(need_revision=False) # 给个默认值

```

### 为什么推荐用 Pydantic？

1. **稳定性**：CrewAI 底层会自动提示 LLM 格式错误并让其重试，成功率远高于我们自己写正则。
2. **代码简洁**：删除了整个 `reviewer_parser.py` 文件及其复杂的正则逻辑。
3. **类型安全**：代码里用 `task.chapter` (属性) 而不是 `task["chapter"]` (字典键)，编辑器能自动提示，也避免了拼写错误。

### 总结建议

如果您想快速修复，请先执行 **第一步（补全 kickoff）** 和 **第二步（清缓存）**。
如果您想一劳永逸，建议采用 **第三步（Pydantic）** 改造代码。

In [ ]:
如果你愿意，下一步我可以直接帮你：

把 “日期 / 区域 / 年份 / 研究对象” 全部做成系统字段白名单

给你一个 ReportSanitizer 模块

或者做一个 Reviewer → System Guardrail Agent

In [ ]:
很好，这一步**非常关键**。下面我直接给你一套**可以原样复制使用的最终实现方案**，严格对应你提出的三点，不引入额外复杂性，**一次性跑通**。

我会按 **文件 → 代码 → 说明** 的顺序来，方便你直接落地。

---

# 一、新建 `log_buffer.py`（Agent 可观测核心）

📁 **路径建议**

```
agent_system/
└── observability/
    ├── __init__.py
    └── log_buffer.py
```

---

## `agent_system/observability/log_buffer.py`

```python
# 用于 Agent 行为可观测（Manus 风格）
from datetime import datetime
from typing import List, Dict

# 全局日志缓冲区（单进程安全）
LOG_BUFFER: List[Dict] = []


def log_event(
    agent: str,
    action: str,
    content: str,
    level: str = "info"
):
    """
    记录 Agent 的一次行为
    """
    LOG_BUFFER.append({
        "time": datetime.now().strftime("%H:%M:%S"),
        "agent": agent,
        "action": action,
        "content": content,
        "level": level
    })


def get_logs() -> List[Dict]:
    """
    前端轮询读取
    """
    return LOG_BUFFER


def clear_logs():
    """
    每次新任务开始前调用
    """
    LOG_BUFFER.clear()
```

📌 设计说明：

* **不用 logging 模块**，避免 Streamlit 多线程问题
* 单进程 + 全局 List = 稳定、好 debug
* 后续可以无痛升级为 WebSocket / Trace

---

# 二、在 `tools_custom.py` 给每个 Tool 加 log_event

下面是**标准改造模式**，你可以照着套。

---

## 示例 1：RAG Tool（最重要）

### 原逻辑（示意）

```python
def rag_tool(query: str) -> str:
    return kb_manager.query_knowledge(query)
```

### ✅ 改造后（推荐直接复制）

```python
from agent_system.observability.log_buffer import log_event
from knowledge_engine import kb_manager

def rag_tool(query: str) -> str:
    log_event(
        agent="Researcher",
        action="RAG 检索",
        content=f"查询问题：{query}"
    )

    result = kb_manager.query_knowledge(query)

    log_event(
        agent="Researcher",
        action="RAG 返回证据",
        content=result[:800] if result else "未检索到有效内容"
    )

    return result
```

---

## 示例 2：搜索工具（Serper / Web）

```python
def serper_tool(query: str) -> str:
    log_event(
        agent="Researcher",
        action="网络搜索",
        content=f"搜索关键词：{query}"
    )

    result = serper_search(query)

    log_event(
        agent="Researcher",
        action="搜索结果摘要",
        content=result[:800]
    )

    return result
```

---

## 示例 3：财务查询 Tool

```python
def stock_analysis(company_name: str) -> str:
    log_event(
        agent="Researcher",
        action="财务数据查询",
        content=f"公司：{company_name}"
    )

    result = query_stock_data(company_name)

    log_event(
        agent="Researcher",
        action="财务数据返回",
        content=result[:800]
    )

    return result
```

📌 原则：

* **Tool 调用前 → log_event**
* **Tool 返回后 → log_event**
* content 永远截断（防止 UI 卡死）

---

# 三、在行业研究启动前清空日志（非常重要）

📄 `industry_research.py`

在 `run_industry_research` 函数开头：

```python
from agent_system.observability.log_buffer import clear_logs

def run_industry_research(inputs):
    clear_logs()  # ⬅️ 每次任务重新开始
    ...
```

否则前端会看到“历史任务残留日志”。

---

# 四、Streamlit 前端增加「🧠 Agent 实时工作流」

📄 **app.py**

在你「📊 行业深度研究」模块中，推荐放在右侧显示区上方。

---

## 1️⃣ 引入日志接口

```python
from agent_system.observability.log_buffer import get_logs
import time
```

---

## 2️⃣ 增加 Manus 风格实时面板

```python
with st.expander("🧠 Agent 实时工作流（可观测）", expanded=True):
    log_container = st.container()
```

---

## 3️⃣ 在点击“生成研报”后，实时刷新

把你原来的按钮逻辑改成如下结构（关键）：

```python
if st.button("🚀 生成深度研报", use_container_width=True):
    if not HAS_BACKEND:
        st.error("无法调用后端，请检查 main.py")
    else:
        log_placeholder = log_container.empty()

        try:
            with st.spinner("多智能体团队正在协作中..."):
                res = main.run_investment_analysis(
                    final_topic,
                    sel_province,
                    int(target_year),
                    focus_prompt
                )

            st.session_state.ind_report = res
            st.success("研报生成完成！")

        except Exception as e:
            st.error(f"运行出错: {e}")

        # === 实时渲染日志（最终展示） ===
        logs = get_logs()
        if logs:
            md = ""
            for l in logs:
                md += f"""
**[{l['time']}] {l['agent']} · {l['action']}**  
{l['content']}

---
"""
            log_placeholder.markdown(md)
```

---

# 五、最终效果（你将看到什么）

前端页面会出现类似👇的效果：

```
🧠 Agent 实时工作流

[14:21:03] Researcher · RAG 检索
查询问题：2024 年 上海 新能源产业 营收

[14:21:07] Researcher · RAG 返回证据
[来源: xxx 研报]
2024 年上海新能源产业产值约 1.2 万亿元，同比增长 11.3%...

[14:21:10] Researcher · 财务数据查询
公司：宁德时代

[14:21:13] Researcher · 财务数据返回
2024 年营收 4000 亿元，净利润 420 亿元，毛利率 25%...
```

👉 **这就是 Manus 风格的核心：**

* 可见搜索
* 可见证据
* 可见 Agent 行为

---

# 六、接下来可以继续升级的方向（可选）

等你确认这一步跑通后，我可以继续帮你做：

1. **分 Agent 颜色区分（Planner / Researcher / Analyst）**
2. **Evidence Panel（只展示“引用证据”）**
3. **每个章节对应证据来源（脚注级别）**
4. **Agent Memory 命中日志（让用户看到“我记住了什么”）**

你现在这一步已经非常接近**真正可用的投研 Agent 系统**了。

你可以直接开始实现；
如果你愿意，下一步我可以**直接帮你改完整 `tools_custom.py` 一版**。


In [ ]:
下面给你一套**可直接落地的「Agent Memory Schema + 向量字段设计」**，是专门为你现在这个**行业研究 / 万字研报 / Agentic RAG**项目设计的，而不是泛泛而谈的“记忆”。

我会按**工程可实现性 + 投研价值密度**来设计。

---

# 一、整体 Memory 分层设计（先给结论）

你这个项目里，**不要搞“一个大向量库”**，而是要 **4 类 Memory，各司其职**：

```
┌──────────────────────────┐
│ ① Research Evidence Memory │  ← 事实 / 数据 / 来源（最重要）
├──────────────────────────┤
│ ② Analytical Insight Memory│  ← 分析结论 / 对比 / 判断
├──────────────────────────┤
│ ③ Report Structure Memory │  ← 章节模板 / 表格结构
├──────────────────────────┤
│ ④ Execution Trace Memory  │  ← Agent 行为轨迹（可选）
└──────────────────────────┘
```

👉 **向量只放“可被复用的高价值片段”**
👉 原始长文本 ≠ 直接入向量库

---

# 二、Memory ①：Research Evidence Memory（事实型记忆）

这是你整个系统**最核心、最值得积累的资产**。

## 1️⃣ 使用场景

* Researcher → Analyst
* Writer 引用历史事实
* 跨行业 / 跨年份复用数据
* 防止重复查同一政策 / 同一统计年鉴

---

## 2️⃣ Schema 设计（强烈建议）

```python
ResearchEvidenceMemory = {
    "memory_id": str,              # uuid
    "industry": str,               # 新能源 / 半导体 / 医疗
    "sub_industry": str | None,    # 光伏 / 储能 / 芯片设计
    "region": str | None,          # 全国 / 江苏 / 广东
    "time_range": str,             # 2022-2024

    "evidence_type": str,          # 财务 / 政策 / 行业统计 / 研报观点
    "entity": str | None,          # 公司名 / 政策名 / 指标名

    "fact_text": str,              # 原子化事实（不超过 300 字）
    "key_numbers": list[str],      # ["2023营收120亿", "CAGR=18%"]

    "source": str,                 # Wind / 年报 / 国家统计局
    "source_level": str,           # 官方 / 券商 / 咨询 / 媒体

    "confidence": float,           # 0.0–1.0（可后处理）
    "created_at": datetime
}
```

---

## 3️⃣ 向量字段设计（关键）

### 向量化字段（embedding）

```text
embedding_text =
f"""
行业: {industry}
区域: {region}
类型: {evidence_type}
事实: {fact_text}
关键数据: {key_numbers}
"""
```

👉 不要 embed 原始 PDF
👉 不要 embed 大段研报
👉 **只 embed “事实级别原子信息”**

---

## 4️⃣ Chroma 建议 collection

```python
collection = chroma_client.get_or_create_collection(
    name="research_evidence_memory",
    metadata={"type": "fact_level"}
)
```

---

# 三、Memory ②：Analytical Insight Memory（分析型记忆）

这是你迈向 **Agentic RAG** 的关键一步。

## 1️⃣ 使用场景

* Analyst 复用「过往行业判断」
* Writer 借鉴类似行业分析逻辑
* Planner 学习“哪些拆解方式有效”

---

## 2️⃣ Schema 设计

```python
AnalyticalInsightMemory = {
    "memory_id": str,

    "industry": str,
    "region": str | None,
    "target_year": int,

    "analysis_type": str,        # 竞争格局 / 产业链缺口 / 估值逻辑
    "conclusion": str,           # 结论性判断（≤200字）

    "supporting_evidence_ids": list[str],  # 引用的 ResearchEvidence

    "investment_implication": str,         # 对投资的直接意义

    "applicable_scenarios": list[str],     # ["VC", "PE", "政府基金"]

    "confidence": float,
    "created_at": datetime
}
```

---

## 3️⃣ 向量字段设计

```text
embedding_text =
f"""
行业: {industry}
分析类型: {analysis_type}
核心结论: {conclusion}
投资含义: {investment_implication}
"""
```

👉 这是「思维级别」的向量
👉 和事实型 memory **严格分库**

---

# 四、Memory ③：Report Structure Memory（结构型记忆）

这个非常重要，但经常被忽略。

## 1️⃣ 用途

* Planner 自动学习「高质量研报结构」
* Writer 复用“章节模板 + 表格样式”
* 稳定万字输出

---

## 2️⃣ Schema 设计

```python
ReportStructureMemory = {
    "memory_id": str,

    "industry": str,
    "report_type": str,          # 行业深度 / 可研 / 投资备忘录

    "chapter_title": str,
    "chapter_goal": str,

    "recommended_word_count": int,
    "table_templates": list[str],  # Markdown 表格骨架

    "usage_count": int,
    "quality_score": float
}
```

---

## 3️⃣ 向量字段（可选，但推荐）

```text
embedding_text =
f"""
行业: {industry}
章节: {chapter_title}
目标: {chapter_goal}
"""
```

---

# 五、Memory ④：Execution Trace Memory（可选，高阶）

这是**真正的 Agentic**，但你可以后面再上。

## 用途

* 记录 Agent 用了哪些 tool
* 哪些搜索结果最终被采用
* 哪些步骤经常失败

```python
ExecutionTraceMemory = {
    "task_id": str,
    "agent_role": str,

    "input_summary": str,
    "tools_used": list[str],

    "success": bool,
    "error_type": str | None,

    "created_at": datetime
}
```

👉 **不向量化**
👉 主要用于系统自我优化

---

# 六、你现在立刻可以落地的「最小可行方案（MVP）」

### ✅ 必做（性价比最高）

| 项                         | 是否必做 |
| ------------------------- | ---- |
| Research Evidence Memory  | ✅    |
| Analytical Insight Memory | ✅    |
| 原子化事实向量                   | ✅    |
| 分 collection              | ✅    |

### ❌ 可以先不做

* Execution Trace 自动反思
* confidence 自动打分
* usage_count 自动学习

---

# 七、你这个项目的“正确演进路线”

```
阶段 1：RAG（你现在）
  → 查资料 + 写报告

阶段 2：Memory-enhanced RAG（你正在做）
  → 查 + 复用过去研究

阶段 3：Agentic RAG（下一步）
  → Planner 会“想起”类似行业怎么拆
  → Analyst 会“引用”历史判断
  → Writer 会“照着成功模板写”
```

---

## 下一步我可以继续帮你做的（强烈推荐）

1️⃣ **Chroma collection 初始化代码（含 metadata 索引）**
2️⃣ **Researcher 自动“事实拆分 → 入库”函数**
3️⃣ **Analyst RAG 检索 prompt（带 memory 引用）**
4️⃣ **Writer 自动引用 memory 的标注格式**

你现在已经不缺 Agent 了，**缺的是「可积累的认知资产结构」**。
这套 Memory Schema，正是为此服务的。


In [3]:
import os
import requests

# 确保这里的端口和你的代理软件一致
proxy = "http://127.0.0.1:15236"
os.environ["http_proxy"] = proxy
os.environ["https_proxy"] = proxy

print(f"正在测试通过代理 {proxy} 连接 Google...")

try:
    # 尝试连接 Google (Gemini 的服务器)
    response = requests.get("https://www.google.com", timeout=10)
    if response.status_code == 200:
        print("✅ 网络连接成功！代理工作正常。")
    else:
        print(f"⚠️ 连接通了，但状态码不对: {response.status_code}")
except Exception as e:
    print(f"❌ 连接失败: {e}")
    print("请检查：1. 代理软件是否开启 2. 端口号 7890 是否正确")

正在测试通过代理 http://127.0.0.1:15236 连接 Google...
✅ 网络连接成功！代理工作正常。


In [ ]:
# Python代码生成ppt
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN

In [ ]:
# --- 配置配色方案 (金融蓝风格) ---
DARK_BLUE = RGBColor(30, 58, 138)  # 主标题色
LIGHT_BLUE = RGBColor(59, 130, 246) # 强调色
GRAY_TEXT = RGBColor(80, 80, 80)    # 正文色

def create_presentation():
    prs = Presentation()

    # --- 辅助函数：设置文本格式 ---
    def set_font(run, size, bold=False, color=GRAY_TEXT):
        run.font.name = 'Arial' # 或 'Microsoft YaHei'
        run.font.size = Pt(size)
        run.font.bold = bold
        run.font.color.rgb = color

    # --- 辅助函数：创建标题幻灯片 ---
    def add_title_slide(title_text, subtitle_text):
        slide = prs.slides.add_slide(prs.slide_layouts[0])
        title = slide.shapes.title
        subtitle = slide.placeholders[1]

        title.text = title_text
        set_font(title.text_frame.paragraphs[0].runs[0], 44, True, DARK_BLUE)
        
        subtitle.text = subtitle_text
        set_font(subtitle.text_frame.paragraphs[0].runs[0], 24, False, LIGHT_BLUE)

    # --- 辅助函数：创建内容幻灯片 ---
    def add_content_slide(title_text, content_list):
        slide = prs.slides.add_slide(prs.slide_layouts[1])
        title = slide.shapes.title
        body = slide.placeholders[1]

        title.text = title_text
        set_font(title.text_frame.paragraphs[0].runs[0], 32, True, DARK_BLUE)

        tf = body.text_frame
        for i, item in enumerate(content_list):
            if i == 0:
                p = tf.paragraphs[0]
            else:
                p = tf.add_paragraph()
            
            p.text = item
            p.space_after = Pt(14)
            set_font(p.runs[0], 20, False, GRAY_TEXT)

    # --- 辅助函数：创建对比表格幻灯片 (竞争优势) ---
    def add_table_slide(title_text, data):
        slide = prs.slides.add_slide(prs.slide_layouts[5]) # Title Only layout
        title = slide.shapes.title
        title.text = title_text
        set_font(title.text_frame.paragraphs[0].runs[0], 32, True, DARK_BLUE)

        # 定义表格位置和大小
        rows = len(data)
        cols = len(data[0])
        left = Inches(0.5)
        top = Inches(1.5)
        width = Inches(9.0)
        height = Inches(0.8 * rows)

        table = slide.shapes.add_table(rows, cols, left, top, width, height).table

        # 设置列宽
        table.columns[0].width = Inches(2.0)
        table.columns[1].width = Inches(2.3)
        table.columns[2].width = Inches(2.3)
        table.columns[3].width = Inches(2.4)

        for i in range(rows):
            for j in range(cols):
                cell = table.cell(i, j)
                cell.text = data[i][j]
                
                # 格式化单元格文本
                run = cell.text_frame.paragraphs[0].runs[0]
                run.font.size = Pt(14)
                run.font.name = 'Arial'
                
                # 表头加粗变色
                if i == 0:
                    run.font.bold = True
                    run.font.color.rgb = DARK_BLUE
                    cell.fill.solid()
                    cell.fill.fore_color.rgb = RGBColor(220, 230, 241)
                # 我们的产品列加粗
                elif j == 3:
                    run.font.bold = True
                    run.font.color.rgb = DARK_BLUE

    # ================= 幻灯片内容生成 =================

    # Slide 1: 封面
    add_title_slide(
        "FinSight AI Pro", 
        "下一代智能投研与决策终端\n\n让 AI 成为您的首席分析师，重塑一级市场投研效率"
    )

    # Slide 2: 痛点
    add_content_slide("痛点与机遇：投研行业的效率危机", [
        "❌ 数据孤岛与碎片化：分析师 70% 时间浪费在搜集数据、整理纪要等“搬砖”工作。",
        "❌ 初级工作重复低效：撰写基础研报、录入财务模型耗时极长且易出错。",
        "❌ 通用 AI 不够专业：ChatGPT 缺乏金融垂类知识，不懂四级产业链，易产生幻觉。",
        "❌ 需求千人千面：政府关注招商，VC关注高增长，传统工具无法灵活切换视角。"
    ])

    # Slide 3: 解决方案
    add_content_slide("解决方案：FinSight AI Pro", [
        "✅ 全流程自动化：大纲规划 -> 异步搜集 -> 逻辑分析 -> 报告撰写 -> 审核，全闭环。",
        "✅ 千人千面定制：支持 G端(政府)、B端(VC)、风控端 多视角切换。",
        "✅ 极速并行处理：异步并行架构，将 3 天的研报工作压缩至 5 分钟。",
        "✅ 深度逻辑穿透：内置“四级产业链”图谱，穿透宏观赛道至微观技术节点。"
    ])

    # Slide 4: 核心技术
    add_content_slide("核心技术壁垒：工程化的智能体系统", [
        "🧠 CrewAI 多智能体架构：Planner(大脑) + Researcher(搜集) + Analyst(逻辑) + Writer(笔杆)。",
        "⚡ 异步并行搜索：突破串行瓶颈，多线程并发获取财务、政策、行业数据。",
        "📚 RAG + 实时联网：本地知识库(私有数据) 与 实时联网(最新数据) 双引擎驱动。",
        "📊 结构化输出：强制生成 Markdown 表格与可视化图表，非单纯文本堆砌。"
    ])

    # Slide 5: 产品功能矩阵
    add_content_slide("产品功能矩阵：金融投研的“瑞士军刀”", [
        "📊 行业深度研究：四级产业链穿透，多视角自动生成深度研报。",
        "⚖️ 智能尽职调查：Growth vs Risk 框架，自动扫描财务粉饰与法律红线。",
        "💰 财务估值建模：自动读取 Pitchbook，一键生成 DCF 模型与 Football Field。",
        "📝 智能会议纪要：音视频转录 + 说话人识别 + Action Item 提取。",
        "📑 BP/财报解读：自动提取 SWOT，计算 KPI，风险预警。"
    ])

    # Slide 6: 竞争优势 (表格)
    table_data = [
        ["维度", "传统投研工具", "通用大模型", "FinSight AI Pro"],
        ["专业度", "仅提供数据，无观点", "易胡说，数据滞后", "数据实时+逻辑严密+自动制表"],
        ["流程", "纯人工操作", "简单问答交互", "Agentic Workflow (自主执行)"],
        ["定制化", "无", "需复杂Prompt工程", "内置专家Prompt，一键切换"],
        ["效率", "3天 / 篇", "需反复对话调整", "5分钟 / 篇 (异步并行)"]
    ]
    add_table_slide("竞争优势：Why Us?", table_data)

    # Slide 7: 商业模式
    add_content_slide("商业模式与市场", [
        "1. SaaS 订阅服务：面向中小型机构，按月/年付费。",
        "2. API / SDK 集成：集成至 Wind、同花顺或券商内部系统，按调用量收费。",
        "3. 私有化部署：面向国资/大型基金，本地化部署，确保数据不出域。"
    ])

    # Slide 8: 结尾
    add_title_slide(
        "FinSight AI Pro",
        "寻求融资 / 合作伙伴\n\n联系我们: [您的名字/联系方式]"
    )

    # 保存文件
    output_filename = "FinSight_AI_Pro_BP.pptx"
    prs.save(output_filename)
    print(f"✅ PPT 已生成：{output_filename}")

if __name__ == "__main__":
    create_presentation()